In [13]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import os

import snappy
from snappy import Product, ProductIO, ProductUtils, WKTReader, HashMap, GPF, jpy

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt


In [3]:
# loading data
productPath='./data/SLCData/S1B_IW_SLC__1SDH_20190530T101600_20190530T101628_016472_01F016_3938.zip'
p = ProductIO.readProduct(productPath)

In [4]:
# look at metadata
def printMetadata(product):
    name = product.getName()
    print("Name: {}".format(name))
    width = product.getSceneRasterWidth()
    print("Width: {} px".format(width))
    height = product.getSceneRasterHeight()
    print("Height: {} px".format(height))
    band_names = product.getBandNames()
    print("Band names: {}".format(",".join(band_names)))

In [5]:
printMetadata(p)

Name: S1B_IW_SLC__1SDH_20190530T101600_20190530T101628_016472_01F016_3938
Width: 73030 px
Height: 15060 px
Band names: i_IW1_HH,q_IW1_HH,Intensity_IW1_HH,i_IW1_HV,q_IW1_HV,Intensity_IW1_HV,i_IW2_HH,q_IW2_HH,Intensity_IW2_HH,i_IW2_HV,q_IW2_HV,Intensity_IW2_HV,i_IW3_HH,q_IW3_HH,Intensity_IW3_HH,i_IW3_HV,q_IW3_HV,Intensity_IW3_HV


In [6]:
def plotBand(product, band):#, vmin, vmax):
    band = product.getBand(band)
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    print(w, h)
    
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0,0,w,h,band_data)
    
    band_data.shape = h, w
    
    width = 12
    height = 12
    plt.figure(figsize=(width,height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary)#, vmin=vmin, vmax=vmax)
    
    return imgplot

## Image pre-processing

### Applying Orbit File

In [7]:
def orbitFileApplication(product):
    parameters = HashMap()

    GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

    parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
    parameters.put('polyDegree', '3')
    parameters.put('continueOnFail', 'false')
    parameters.put('copyMetadata', True)

    apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, product)
    
    return apply_orbit_file

In [8]:
pOrbit = orbitFileApplication(p)

### Clipping/Subsetting images

In [9]:
outline = geojson_to_wkt(read_geojson(r'../inData/diskoIsland.geojson'))

In [10]:
def clipImages(orbitFile, bounds):
    
    geometry = WKTReader().read(bounds) 
    
    SubsetOp = snappy.jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
    GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
    
    parameters = HashMap()
    parameters.put('copyMetadata', True)
    parameters.put('geoRegion', geometry)
    
    product_subset = snappy.GPF.createProduct('Subset', parameters, orbitFile)
    
    return product_subset

In [11]:
subset = clipImages(pOrbit,outline)

In [16]:
printMetadata(subset)

Name: Subset_S1B_IW_SLC__1SDH_20190530T101600_20190530T101628_016472_01F016_3938_Orb
Width: 34701 px
Height: 9639 px
Band names: i_IW1_HH,q_IW1_HH,Intensity_IW1_HH,i_IW1_HV,q_IW1_HV,Intensity_IW1_HV,i_IW2_HH,q_IW2_HH,Intensity_IW2_HH,i_IW2_HV,q_IW2_HV,Intensity_IW2_HV,i_IW3_HH,q_IW3_HH,Intensity_IW3_HH,i_IW3_HV,q_IW3_HV,Intensity_IW3_HV


In [14]:
# progress monitor for java writer
def createProgressMonitor():
    PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem = jpy.get_type('java.lang.System')
    monitor = PWPM(JavaSystem.out)
    return monitor
 
pm = createProgressMonitor() 

In [19]:
# write product

# options
ProductIO.writeProduct(subset, subset.getName()+".dim", "BEAM-DIMAP")

# Alternative solution: Computations are faster when using GPF to write the product instead of ProductIO:
#incremental = False # most writer don't support the incremental writing mode (update exsiting file), except BEAM-DIMAP.

#GPF.writeProduct(subset , './data/processed/', 'BEAM-DIMAP', incremental, pm)

# without progress monitor 
#GPF.writeProduct(target_product , File(<'your/out/directory'>), write_format, incremental, ProgressMonitor.NULL)


In [11]:
#plotBand(subset, 'Intensity_IW3_HH')

# note: write multiple bands
BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand1 = BandDescriptor()
targetBand1.name = 'band_1'
targetBand1.type = 'float32'
targetBand1.expression = '(radiance_10 - radiance_7) / (radiance_10 + radiance_7)'

targetBand2 = BandDescriptor()
targetBand2.name = 'band_2'
targetBand2.type = 'float32'
targetBand2.expression = '(radiance_9 - radiance_6) / (radiance_9 + radiance_6)'

targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 2)
targetBands[0] = targetBand1
targetBands[1] = targetBand2

parameters = HashMap()
parameters.put('targetBands', targetBands)

result = GPF.createProduct('BandMaths', parameters, product)

In [12]:
#ReprojectOp = snappy.jpy.get_type('org.esa.snap.core.gpf.common.reproject.ReprojectionOp')

In [13]:
#list(p.getBandNames())

# create subset of data


wkt=a“POLYGON((7.62 81.585, 7.62 81.699, 7.465 81.699, 7.465 81.585,7.62 81.585 ))”
geom = WKTReader().read(wkt)
HashMap = jpy.get_type(‘java.util.HashMap’)
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
parameters = HashMap()
parameters.put(‘copyMetadata’, True)
parameters.put(‘geoRegion’, geom)
SAR_image_subset = GPF.createProduct(‘Subset’, parameters, SAR_image)
ProductIO.writeProduct(SAR_image_subset, image_name+’_subset’, ‘BEAM-DIMAP’)